In [ ]:
!pip install transformers datasets accelerate matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
#IMPORTS:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt
import random

In [ ]:
#DEFINITIONS:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [ ]:
!pip install -U fsspec datasets


  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
drive.mount('/content/drive')
# Lroad dataset
squad = load_dataset("squad")

# Load tokenizer and model
#tokenizer = DistilBertTokenizerFast.from_pretrained("/content/drive/My Drive/Distil-BERT/QA/Model5")
#model = DistilBertForQuestionAnswering.from_pretrained("/content/drive/My Drive/Distil-BERT/QA/Model5")
tokenizer = DistilBertTokenizerFast.from_pretrained("/content/drive/My Drive/Distil-BERT/1/QA/Model10")
model = DistilBertForQuestionAnswering.from_pretrained("/content/drive/My Drive/Distil-BERT/1/QA/Model10")

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training
model.train()
save_every = 4
for count in range(11,20):
    seed1 = random.randint(1,1000)
    batch_num = 0
    totalLoss = 0;
    LossVector = []
    for epoch in range(3):
        small_train = squad["train"].shuffle(seed=seed1).select(range(1000))

    # Tokenize the reduced dataset
        tokenized_squad = small_train.map(
            prepare_train_features,
            batched=True,
            remove_columns=small_train.column_names
        )


    # DataLoader
        train_loader = DataLoader(tokenized_squad, batch_size=16, shuffle=True, collate_fn=default_data_collator)
        progress = tqdm(train_loader)
        for batch in progress:
            batch_num += 1

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                start_positions=start_positions,
                end_positions=end_positions,
            )

            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            totalLoss += loss.item()
            LossVector.append(loss.item())
            avg = totalLoss / batch_num

            progress.set_description(f"Loss: {avg:.4f}")
    count += 1

    save_path = f"/content/drive/My Drive/Distil-BERT/1/QA/Model1" + str(count)                     #
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model saved at {save_path}")
    print(f"Model {count} Loss Vector: {LossVector}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loss: 0.4681: 100%|██████████| 64/64 [33:32<00:00, 31.45s/it]


Model saved at /content/drive/My Drive/Distil-BERT/1/QA/Model113
Model 13 Loss Vector: [1.309157133102417, 1.0718798637390137, 1.512460708618164, 1.830876350402832, 1.05655038356781, 0.432056188583374, 1.2501423358917236, 0.5042868852615356, 2.8633875846862793, 0.6526779532432556, 1.210655927658081, 0.47277796268463135, 0.5492513179779053, 0.8096470832824707, 1.429279088973999, 0.9173486232757568, 1.1195034980773926, 0.5523338913917542, 1.4805166721343994, 0.674156904220581, 0.9287045001983643, 0.7995373010635376, 0.9720795154571533, 1.0244536399841309, 1.048462986946106, 0.5521015524864197, 0.4232420325279236, 1.41062593460083, 1.021301507949829, 0.9834004044532776, 0.7621690034866333, 0.76528000831604, 1.0988407135009766, 0.8648825883865356, 0.7002019286155701, 0.9104287028312683, 1.9617029428482056, 0.6446971893310547, 1.0083494186401367, 1.2156124114990234, 1.005932331085205, 0.6589380502700806, 0.5713213682174683, 0.9524326324462891, 1.029294490814209, 0.7521887421607971, 0.547567

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loss: 0.5176:  97%|█████████▋| 62/64 [32:15<01:02, 31.19s/it]

In [ ]:
LossVector = [0.6036608815193176, 0.6953475475311279, 0.7122277617454529, 0.49004414677619934, 0.7173977494239807, 0.9640514254570007, 0.5375427007675171, 0.6558915376663208, 1.0949697494506836, 0.8975297808647156, 1.2526321411132812, 0.7718744277954102, 1.3182440996170044, 1.7401280403137207, 0.7599726915359497, 0.6013151407241821, 0.4307655692100525, 0.4270564317703247, 1.0358233451843262, 1.2077698707580566, 0.7430063486099243, 0.5652590990066528, 0.9932474493980408, 0.8813902735710144, 1.2397098541259766, 0.8389221429824829, 0.39747288823127747, 1.054903268814087, 1.339782476425171, 0.8879643678665161, 0.6416682004928589, 0.8966537714004517, 0.6114922761917114, 0.718422532081604, 1.2417054176330566, 1.1247704029083252, 1.1795876026153564, 1.0780494213104248, 1.154346227645874, 1.082876443862915, 1.0130399465560913, 0.6444147825241089, 0.7304782271385193, 1.1583104133605957, 0.4361444413661957, 0.9670709371566772, 0.980726957321167, 0.6361804008483887, 1.2891303300857544, 0.7633465528488159, 1.094005823135376, 0.34425774216651917, 0.4800799489021301, 0.8811005353927612, 1.2497906684875488, 1.5629220008850098, 1.4511189460754395, 0.8260108828544617, 1.255053997039795, 0.8622468113899231, 0.8879363536834717, 0.9622023105621338, 0.8847346305847168, 1.6970385313034058, 0.4575049877166748, 0.3987078070640564, 0.3915339708328247, 0.3382652699947357, 0.38410770893096924, 0.4905495047569275, 0.49065476655960083, 0.452473521232605, 0.4358404278755188, 0.2797497510910034, 0.2368067502975464, 0.15867993235588074, 0.36099493503570557, 0.31767356395721436, 0.33312520384788513, 0.38278791308403015, 0.7012206315994263, 0.2673179507255554, 0.5791696906089783, 0.5061836242675781, 0.41085729002952576, 0.28462502360343933, 0.41258326172828674, 0.36640840768814087, 0.4623205065727234, 0.2968665361404419, 0.13128948211669922, 0.3778741955757141, 0.30300530791282654, 0.42903071641921997, 0.26102250814437866, 0.17157919704914093, 0.11795918643474579, 0.263751745223999, 0.5980192422866821, 0.2414746880531311, 0.28289973735809326, 0.548733651638031, 0.5147023797035217, 0.11191031336784363, 0.3571411073207855, 0.8444629311561584, 0.25963231921195984, 0.2518996000289917, 0.4495088458061218, 0.2951429486274719, 0.17879918217658997, 0.2794361114501953, 0.27994704246520996, 0.2802671194076538, 0.30316662788391113, 0.16090568900108337, 0.22962120175361633, 0.31194692850112915, 0.43380647897720337, 0.3473939299583435, 0.17952629923820496, 1.2040269374847412, 0.8801445960998535, 0.6477779150009155, 0.4117611348628998, 0.3004496991634369, 0.2087767869234085, 0.6374943256378174, 0.20056548714637756, 0.15308785438537598, 0.13772015273571014, 0.13863202929496765, 0.49411243200302124, 0.2170361876487732, 0.2465311586856842, 0.22425687313079834, 0.1925310492515564, 0.1915970742702484, 0.29949384927749634, 0.2025035321712494, 0.4401838183403015, 0.15101288259029388, 0.25467851758003235, 0.12059995532035828, 0.16846846044063568, 0.25278007984161377, 0.07854688912630081, 0.19574850797653198, 0.2689204812049866, 0.0927601158618927, 0.2050771415233612, 0.10698144137859344, 0.16198910772800446, 0.3462620675563812, 0.16364111006259918, 0.11820977181196213, 0.17955738306045532, 0.1286560297012329, 0.2892839312553406, 0.20717166364192963, 0.08317384123802185, 0.13375456631183624, 0.37405550479888916, 0.1720830202102661, 0.2534913420677185, 0.16007646918296814, 0.15786798298358917, 0.08030769228935242, 0.07244593650102615, 0.09967456012964249, 0.2211175113916397, 0.15676498413085938, 0.157626673579216, 0.130857914686203, 0.12787380814552307, 0.17799146473407745, 0.18740662932395935, 0.21788036823272705, 0.2989838421344757, 0.31920233368873596, 0.3123314380645752, 0.20129375159740448, 0.18367941677570343, 0.07809656858444214, 0.2662064731121063, 0.23638221621513367, 0.09571833163499832, 0.23180243372917175, 0.11208193004131317, 0.08255960792303085, 0.34202075004577637, 0.05818464607000351]

plt.figure(figsize=(10, 6))
plt.plot(range(len(LossVector)), LossVector, label="Loss Over Time", marker='o')  # <- Fixed this line
plt.title("Loss Graph")
plt.xlabel("Batch Step")
plt.ylabel("Loss")
plt.ylim(0, max(LossVector) + 0.1)  # Dynamic y-limit
plt.legend()
plt.grid(True)
plt.show()